# 🐦 BirdNET Embedding Classifier (3s Chunks)
Using CNN + TensorFlow + RTX 3050 Acceleration

In [3]:
# 📦 Step 1: Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models # type: ignore

In [4]:
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU devices:", tf.config.list_physical_devices('GPU'))

TF version: 2.15.0
Built with CUDA: False
GPU devices: []


In [ ]:
# 📁 Step 2: Load Data
df = pd.read_csv('embeddings_csv/all_embeddings.csv')
X = df[[f'em_{i}' for i in range(1024)]].values
y = df['label'].values

In [ ]:
# 🔤 Step 3: Encode Labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)

In [ ]:
# 🔄 Step 4: Reshape Features for CNN
X_reshaped = X.reshape((-1, 32, 32, 1))

In [ ]:
# 🧪 Step 5: Split Data
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [ ]:
# 🧠 Step 6: Define CNN Model
model = models.Sequential([
    layers.Input(shape=(32, 32, 1)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# 🚀 Step 7: Train Model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))

In [ ]:
# 📤 Step 8: Export Predictions for Leaderboard
sample_submission = pd.read_csv('sample_submission.csv')
predictions = model.predict(X_reshaped)  # Or use X_val
pred_labels = le.inverse_transform(np.argmax(predictions, axis=1))

# Replace all columns with 0s
output = sample_submission.copy()
output.iloc[:, 1:] = 0

# Fill with confident prediction in the correct class column
for i, pred in enumerate(predictions):
    if i >= len(output): break
    top_class = le.inverse_transform([np.argmax(pred)])[0]
    if top_class in output.columns:
        output.loc[i, top_class] = 1.0

# Save CSV
output.to_csv('my_submission.csv', index=False)